Feature engineering
-------------------

Compare the results of MLPs on the validation data after running MLP_nowcasting.py (used for estimating power quality variables). 
Use the best models to estimate 'TDU', 'ITD', 'Q1act' (all three phases 'L1', 'L2', 'L3') and then, to generate deviations.

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import sys
import ast

In [2]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly
from sklearn.linear_model import LinearRegression

In [4]:
from abc import ABC, abstractmethod
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

In [6]:
import tensorflow.keras.backend as K

def metric_IA(y_true,y_pred):
    sse = K.sum(K.square(y_true - y_pred))
    ia = K.sum(K.square(K.abs(y_true - K.mean(y_true)) + K.abs(y_pred - K.mean(y_true))))

    ia = 1. - sse/ia
    return ia

In [2]:
# Quality metrics
def IA(true, predicted):  
    true = true.ravel()
    predicted = predicted.ravel()
    return 1 - np.sum((true-predicted)**2)/np.sum((np.abs(predicted-np.mean(true))+np.abs(true-np.mean(true)))**2)

def RMSE(true, predicted):
    true = true.ravel()
    predicted = predicted.ravel()
    return np.sqrt(np.mean((true-predicted)**2))

def MAE(true, predicted):
    true = true.ravel()
    predicted = predicted.ravel()
    return np.mean(np.abs(true-predicted))

def BIAS(true, predicted):
    true = true.ravel()
    predicted = predicted.ravel()
    return np.mean(predicted-true)

In [7]:
# In our study, there are two experiments: for the ventilation system and for the main distribution board
# Here we initialize a variable "exp_name", which is used to define a path to the measurement data

exp_name = 'main_distribution_board_2018_2020' # or exp_name = 'ventilation_system_2018_2020'

In [8]:
phases = ['L1', 'L2', 'L3']
pq_variables = ['TDU', 'ITD', 'Q1act'] # Variables estimated in nowcasting

In [9]:
# Dictionary to store the best MLP architectures and the number of epochs
best_mlp_settings = dict()

exp_runs = 10

for pq_variable in pq_variables:
    best_mlp_settings[pq_variable] = dict()
    
    for phase in phases:
        best_mlp_settings[pq_variable][phase] = dict()
        
        results = pd.DataFrame()

        for i in range(exp_runs):
            result = pd.read_csv("{}/results_mlp_{}_{}_run_{}.csv".format(exp_name, pq_variable, phase, i), index_col = 0)
            results = results.append(result, ignore_index = True)
          
        summary_avr = results.groupby(['nneurons']).mean()
        for metric in ['rmse', 'mae', 'ia']:
            best_nn = summary_avr.index[np.argmin(summary_avr['{}_valid'.format(metric)])]
            epochs = summary_avr.epochs[np.argmin(summary_avr['{}_valid'.format(metric)])]
            
            best_mlp_settings[pq_variable][phase][metric] = (ast.literal_eval(best_nn), epochs) 

In [7]:
# Mehods to run MLP with the best settings found

class NeuralNet(ABC):
    MAX_EPOCHS = 1000
    BATCH_SIZE = 128
    LOSS = 'mean_absolute_error'  
    METRICS = [metric_IA, 'mean_squared_error']
    LEARNING_RATE = 1e-3
    ACTIVATION = 'tanh'
    NNEURONS = [8]

    def __init__(self, **kwargs):
        self.batch_size = kwargs.get('batch_size', NeuralNet.BATCH_SIZE)
        self.loss = kwargs.get('loss', NeuralNet.LOSS)
        self.metrics = kwargs.get('metrics', NeuralNet.METRICS)
        self.max_epochs = kwargs.get('metrics', NeuralNet.MAX_EPOCHS)
        self.learning_rate = kwargs.get('learning_rate', NeuralNet.LEARNING_RATE)
        self.activation = kwargs.get('activation', NeuralNet.ACTIVATION)
        self.nneurons = kwargs.get('nneurons', NeuralNet.NNEURONS[:])

        self.model = Sequential()
        #self.random_weights = None # used to re-initialize the model in new runs


    @abstractmethod
    def build_model(self):
        pass

    def train_model_with_valid(self, data_train_x, data_train_y, data_val_x, data_val_y):
        '''
        A training mode to define an optimal number of epochs using validation data.
        Returns a tuple (opt_epochs, opt_val_loss)
        '''

        #self.model.set_weights(self.random_weights) # re-initialize the model
        
        opt = Adam(learning_rate=self.learning_rate)
        
        self.model.compile(
            loss = self.loss,
            optimizer = opt,
            metrics =  self.metrics
        )
        
        earlystopping = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=100, verbose=0, mode='min', restore_best_weights=True)
        
        self.model.fit(
            data_train_x, data_train_y,
            verbose=0,
            shuffle=True,
            batch_size=self.batch_size,
            epochs=self.max_epochs,
            validation_data=(data_val_x, data_val_y),
            callbacks=[earlystopping]
        )
        
        hist = self.model.history.history['val_loss']
        n_epochs_best = np.argmin(hist)
        scores_validation = self.model.evaluate(data_val_x, data_val_y, verbose=0, batch_size=data_val_x.shape[0])

        return n_epochs_best
    
    def train_model(self, data_x, data_y, opt_epochs):
        '''
        A training mode without validation when the optimal number of epochs is known.
        '''
        #self.model.set_weights(self.random_weights) # re-initialize the model
        
        opt = Adam(learning_rate=self.learning_rate)
        self.model.compile(
            loss = self.loss,
            optimizer = opt,
            metrics =  self.metrics
        )
        self.model.fit(
            data_x, data_y,
            verbose=0,
            shuffle=True,
            batch_size=self.batch_size,
            epochs=int(opt_epochs),
            validation_split = 0.0
        )

    def predict(self, data_x):
        return self.model.predict(data_x, batch_size = data_x.shape[0])
            

class MLP(NeuralNet):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def build_model(self, input_cols):
        self.model = Sequential()
        for i, nn in enumerate(self.nneurons):
            if i == 0:
                self.model.add(Dense(nn, input_shape=(input_cols,), activation = self.activation))
            else:
                self.model.add(Dense(nn, activation = self.activation))
        self.model.add(Dense(1, activation = 'linear'))

        #self.random_weights = self.model.get_weights() # save random weights to re-initialize the model in new runs

In [8]:
# This function is used to train MLP with the predefided settings on the given training and test data 

def modeling(training_data, test_data, inputs, output, NN, opt_epochs):
    
    training_data_inputs = training_data.loc[:, inputs].copy()
    test_data_inputs = test_data.loc[:, inputs].copy()

    training_data_output = pd.DataFrame(training_data.loc[:, output].copy(), columns = [output])
    test_data_output = pd.DataFrame(test_data.loc[:, output].copy(), columns = [output])
    
    # 1. Train scalers
    scaler_inputs = MinMaxScaler()
    scaler_inputs.fit(training_data_inputs)
    
    training_data_inputs_scaled = scaler_inputs.transform(training_data_inputs)
    test_data_inputs_scaled = scaler_inputs.transform(test_data_inputs)
    #---------------------------------------------------------------------
    scaler_output = MinMaxScaler()
    scaler_output.fit(training_data_output)
    
    training_data_output_scaled = scaler_output.transform(training_data_output)
    test_data_output_scaled = scaler_output.transform(test_data_output)
    #---------------------------------------------------------------------
    
    # 2. Training and test performance
    
    NN.build_model(training_data_inputs_scaled.shape[1])
    NN.train_model(training_data_inputs_scaled, training_data_output_scaled.ravel(), opt_epochs)
    
    pred_train = NN.predict(training_data_inputs_scaled).reshape(-1, 1)
    
    pred_train = scaler_output.inverse_transform(pred_train)
    training_data_output_scaled = scaler_output.inverse_transform(training_data_output_scaled)
    
    ia_train = IA(training_data_output_scaled, pred_train)
    rmse_train = RMSE(training_data_output_scaled, pred_train)
    mae_train = MAE(training_data_output_scaled, pred_train)
    bias_train = BIAS(training_data_output_scaled, pred_train)
    
    pred_test = NN.predict(test_data_inputs_scaled).reshape(-1, 1)
    pred_test = scaler_output.inverse_transform(pred_test)

    test_data_output = scaler_output.inverse_transform(test_data_output_scaled)
    
    ia_test = IA(test_data_output, pred_test)
    rmse_test = RMSE(test_data_output, pred_test)
    mae_test = MAE(test_data_output, pred_test)
    bias_test = BIAS(test_data_output, pred_test)
    
    
    result = {"ia_train": ia_train, "rmse_train": rmse_train, "mae_train": mae_train, "bias_train": bias_train, 
              "ia_test": ia_test, "rmse_test": rmse_test, "mae_test": mae_test, "bias_test": bias_test}
    
    pred_train = pd.DataFrame(pred_train, index = training_data.index, columns = ["predictions"])
    pred_test = pd.DataFrame(pred_test, index = test_data.index, columns = ["predictions"])

    return pred_train, pred_test, result

In [20]:
# Create html to visualize the results of nowcasting

def visualize(pq_variable, phase, nneurons, results, test_year):
    
    fig = make_subplots(rows=2, cols=4, 
                        specs=[[{'colspan': 3}, {}, {}, {}],[{'colspan': 3}, {}, {}, {}]],
                        subplot_titles = [
        'Output {}, phase {}: Training data. RMSE = {}, MAE = {}, IA = {}, BIAS = {}'.format(pq_variable, phase,
                                                                                  round(results['rmse_train'], 2), 
                                                                                  round(results['mae_train'], 2), 
                                                                                  round(results['ia_train'], 2),
                                                                                  round(results['bias_train'], 2)),
                                                                                  None, None,
                                                                                  'Training data',
        'Output {}, phase {}: Test data. RMSE = {}, MAE = {}, IA = {}, BIAS = {}'.format(pq_variable, phase,
                                                                              round(results['rmse_test'], 2), 
                                                                              round(results['mae_test'], 2), 
                                                                              round(results['ia_test'], 2),
                                                                              round(results['bias_test'], 2)),
                                                                              None, None,
                                                                              'Test data'])

    fig.add_trace(go.Scatter(x=pd.to_datetime(training_data.Timestamp), 
                             y=training_data.loc[:, output], mode='markers', marker=dict(size=5),
                             name='measured', legendgroup = '1'), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(predictions_train.Timestamp), marker=dict(size=5),
                             y=predictions_train.loc[:, 'predictions'], mode='markers',
                             name='predicted', legendgroup = '1'), row=1, col=1)
    fig.add_trace(go.Scatter(x=training_data.loc[:, output], 
                             y=predictions_train.loc[:, 'predictions'], mode='markers', marker=dict(size=3),
                             showlegend = False, legendgroup = '1'), row=1, col=4)
    fig.add_trace(go.Scatter(x=[min(training_data.loc[:, output].min(), predictions_train.loc[:, 'predictions'].min()), 
                                max(training_data.loc[:, output].max(), predictions_train.loc[:, 'predictions'].max())],
                             y=[min(training_data.loc[:, output].min(), predictions_train.loc[:, 'predictions'].min()), 
                                max(training_data.loc[:, output].max(), predictions_train.loc[:, 'predictions'].max())],
                            mode="lines",
                            line=go.scatter.Line(color="blue", dash = "dot", width = 2),
                            name="Ref. line: y = x", legendgroup = '1'), row=1, col=4)

    x = training_data.loc[:, [output]].dropna()
    y = predictions_train.loc[:, 'predictions'].dropna()
    common_index = list(set(x.index).intersection(set(y.index)))
    
    reg = LinearRegression().fit(x.loc[common_index], y.loc[common_index])    
    fig.add_trace(go.Scatter(x=[training_data.loc[:, output].min(), training_data.loc[:, output].max()],
                             y=[training_data.loc[:, output].min()*reg.coef_[0] + reg.intercept_, 
                                training_data.loc[:, output].max()*reg.coef_[0] + reg.intercept_],
                            mode="lines",
                            line=go.scatter.Line(color="red", dash = "dot", width = 2),
                            name="OLS line: y = {} * x + {}".format(round(reg.coef_[0], 3), round(reg.intercept_,3)),
                            legendgroup = '1', legendgrouptitle_text='Training data'), 
                            row=1, col=4)


    fig.add_trace(go.Scatter(x=pd.to_datetime(test_data.Timestamp), 
                             y=test_data.loc[:, output], mode='markers', marker=dict(size=5),
                             name='measured', legendgroup = '2'), row=2, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(predictions_test.Timestamp), 
                             y=predictions_test.loc[:, 'predictions'], mode='markers', marker=dict(size=5),
                             name='predicted', legendgroup = '2'), row=2, col=1)
    fig.add_trace(go.Scatter(x=test_data.loc[:, output], 
                             y=predictions_test.loc[:, 'predictions'], mode='markers', marker=dict(size=2),
                             showlegend = False, legendgroup = '2'), row=2, col=4)
    fig.add_trace(go.Scatter(x=[min(test_data.loc[:, output].min(), predictions_test.loc[:, 'predictions'].min()), 
                                max(test_data.loc[:, output].max(), predictions_test.loc[:, 'predictions'].max())],
                             y=[min(test_data.loc[:, output].min(), predictions_test.loc[:, 'predictions'].min()), 
                                max(test_data.loc[:, output].max(), predictions_test.loc[:, 'predictions'].max())],
                            mode="lines",
                            line=go.scatter.Line(color="blue", width = 2),
                            name = "Ref. line: y = x", legendgroup = '2'), row=2, col=4)
    
    x = test_data.loc[:, [output]].dropna()
    y = predictions_test.loc[:, 'predictions'].dropna()
    common_index = list(set(x.index).intersection(set(y.index)))
    
    reg = LinearRegression().fit(x.loc[common_index], y.loc[common_index])    
    fig.add_trace(go.Scatter(x=[test_data.loc[:, output].min(), test_data.loc[:, output].max()],
                             y=[test_data.loc[:, output].min()*reg.coef_[0] + reg.intercept_, 
                                test_data.loc[:, output].max()*reg.coef_[0] + reg.intercept_],
                            mode="lines",
                            line=go.scatter.Line(color="red", width = 2),
                            name="OLS line: y = {} * x + {}".format(round(reg.coef_[0], 3), round(reg.intercept_,3)),
                            legendgroup = '2', legendgrouptitle_text='Test data'), 
                            row=2, col=4)

    fig.update_layout(showlegend=True, title = '{}, neurons = {}'.format(folder, nneurons), legend_tracegroupgap = 10)
    
    fig.update_layout(
    xaxis8_title="Measured",
    yaxis8_title="Predicted",
    yaxis8={'side': 'right'}  )

    #fig.show()

    plotly.offline.plot(fig, filename='{}/TEST {} {} {}.html'.format(exp_name, test_year, pq_variable, phase))

In [21]:
# Save measurements, estimations, and deviations for a particular PQ variable in one file 

def save_predictions(exp_name, pq_variable, phase, test_year):
    
    df = pd.DataFrame(pd.to_datetime(test_data.Timestamp))
    df['measured'] = test_data.loc[:, output]
    df['model outcome'] = predictions_test.loc[:, 'predictions']
    df['deviation (measured - model)'] = df['measured'] - df['model outcome']
    
    df.to_csv('{}/PQ {} {} {}.csv'.format(exp_name, pq_variable, phase, test_year))

    return df
    

In [22]:
#---------------------------------------------
# Run feature engineering for different years
#---------------------------------------------

training_data = pd.read_csv('{}/2018_2019_winter_10min.csv'.format(exp_name))

# Use one of the test years 2019_2020, 2020_2021, or 2021_2022 to generate new features
test_year = '2019_2020'
test_data = pd.read_csv('{}/{}_winter_10min.csv'.format(exp_name, test_year))

training_data_noNaN = training_data.dropna().copy()
test_data_noNaN = test_data.dropna().copy()

column_names = training_data.columns

column_names_L1 = [var for var in column_names if 'L1' in var ]
column_names_L2 = [var for var in column_names if 'L2' in var ]
column_names_L3 = [var for var in column_names if 'L3' in var ]

column_names_dict = {'L1': column_names_L1,
                     'L2': column_names_L2,
                     'L3': column_names_L3}

training_timestamps = training_data_noNaN.loc[:, 'Timestamp']

for pq_variable in pq_variables:
    for phase in phases:
        
        for exp_run in range(1):
            
            output = pq_variable + phase
            inputs = column_names_dict[phase].copy()
            inputs.remove(output)
            inputs.remove('Q1{}'.format(phase))
            inputs.remove('I1{}'.format(phase))
            inputs.append('U0U1')
            inputs.append('U2U1')
            inputs.append('Freq')

            predictions_train = pd.DataFrame(training_data.loc[:, "Timestamp"].copy(), index = training_data.index)
            predictions_test = pd.DataFrame(test_data.loc[:, "Timestamp"].copy(), index = test_data.index)

            print(pq_variable, ' ', phase)

            nneurons = best_mlp_settings[pq_variable][phase]['mae'][0]
            epochs = best_mlp_settings[pq_variable][phase]['mae'][1]

            print(nneurons)

            model = MLP(nneurons = nneurons) 
            pred_train, pred_test, result = modeling(training_data_noNaN, test_data_noNaN, inputs, output, model, epochs)

            predictions_train = predictions_train.join(pred_train)
            predictions_test = predictions_test.join(pred_test)

            saved_df = save_predictions(exp_name, pq_variable, phase, test_year)
            
            # Uncomment, if visualization is needed
            # visualize(pq_variable, phase, nneurons, result, test_year)

TDU   L1
[128, 64, 32]
TDU   L2
[128, 128, 128]
TDU   L3
[128, 128, 128]
ITD   L1
[64, 32, 16]
ITD   L2
[128, 128, 128]
ITD   L3
[128, 128, 128]
Q1act   L1
[64, 32, 16]
Q1act   L2
[16, 16, 16]
Q1act   L3
[128, 128, 128]
